In [1]:
import numpy as np
import scipy
from mpmath import *
import cmath

In [2]:
import math
def normal_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

In [3]:
def F(x, y, t, r, gamma):
    v = (1 - 4 * r)** 0.5    
    w = (gamma * (1 - 2 * r) - 2 * r) / (2 * gamma * (1 - 4 * r)** 0.5)    
    gg = (1 - 4 * r)** 0.5 / gamma
    u = (1 - y) * np.exp(-gamma * t)
    u0 = (1 - y)
    Q = 1 + 2 * w - gg * u0 + (2 * r * (x - y) + y - 1) / gamma
    C = (-Q * whitm(w, 0, gg * u0) + (1 + 2 * w) * whitm(1 + w, 0, gg * u0)) / \
        (Q * whitw(w, 0, gg * u0) + 2 * whitw(1 + w, 0, gg * u0))
    f = 1 - u + \
        (u * (1 + v) - gamma * (1 + 2 * w)) / (2 * r) + \
        (gamma / (2 * r)) * ((1 + 2 * w) * whitm(1 + w, 0, u * gg) -
                             2 * C * whitw(1 + w, 0, u * gg)) / \
        (whitm(w, 0, u * gg) + C * whitw(w, 0, u * gg))
    return f

def Antal(z, t, r, gamma):
    g = []
    for k in range(len(z)):
        if z[k] == 1:
            g.append(1)
        else:
            g.append(F(z[k], z[k], t, r, gamma))
    return g


In [4]:
def Psurv(t, l, r, rho):
    rho = rho - 0.00001
    gamma = rho / (1 - rho)
    T = l * t
    p = 1 - np.real(Antal([0], T, r, gamma))
#     if (p <= 0 or p > 1 or np.isnan(p)):
    if (p <= 0 or p > 1):
        p = Psurv(t, l, r + 0.00001, rho - 0.001)
    return p


In [5]:
def Pn(n, t, l, r, rho, N, returnConsecutive='', max_n=''):
    if max(n) >= N:
        N = max(n) + 1
    rho = rho - 0.00001
    assert (rho < 1)
    assert (rho > 0)
    gamma = rho / (1 - rho)
    T = l * t
#     k = range(N)
#     Gvals = Antal(cmath.exp(2 * np.pi * 1j * k / N), T, r, gamma)
    Gvals = Antal([cmath.exp(2 * np.pi * 1j * k / N) for k in range(int(N))], T, r, gamma)
    p = []
    if returnConsecutive:
        p = np.zeros(max_n)
#         for lp in range(1, len(n) + 1):
        for lp in range(len(n)):
#              p[n[lp]] = \
#                 np.real((1 / N) *
#                         sum(Gvals * np.exp(-2 * np.pi * 1j * k * n[lp] / N)))
            temporary = [cmath.exp(-2 * np.pi * 1j * k * n[lp] / N) for k in range(int(N))]
            p[n[lp]-1] = np.real((1 / N) *sum([a*b for a,b in zip(Gvals,temporary)]))
                               
    else:
#         for lp in range(1, len(n) + 1):
        for lp in range(len(n)):
#             p[lp] = \
#                 np.real((1 / N) * sum(Gvals *
#                                       np.exp(-2 * np.pi * 1j * k * n[lp] / N)))          
            Temporary = [cmath.exp(-2 * np.pi * 1j * k * n[lp] / N) for k in range(int(N))]
            p[lp] = np.real((1 / N) * sum(a*b for a,b in zip(Gvals,Temporary)))
#     print(Gvals)
    tst = sum(p)
#     print(tst)
    if (tst > 1 or tst < 0 or np.isnan(tst)):
        print('Pathological point: r=' + str(r) + ' rho=' + str(
            rho) + '. Making 0.1% perturbation')
        p = Pn(n, t, l, r + 0.00001, rho - 0.001, N, returnConsecutive, max_n)

#     for x in range(len(p)):
#         if (p[x] < 0):
#             p[x] = 0

#     p[p < 0 & p > -1e-5] = 0

    return(p)

In [8]:
def main():
    t = [10 / 7, 30 / 7, 84 / 7, 26]

    nRange = [list(range(1, 8)),
              list(range(1, 14)),
              list(range(1, 27)),
              list(range(1, 56))]

    lambdaRange = [0.4,0.5]
    rhoRange = [0.5,0.75]
    rRange = [0.06,0.08]

    maxN = max(list(map(max, nRange)))
    
#     PScanPP = np.zeros(len(lambdaRange), len(rhoRange), len(rRange),
#                        len(nRange), maxN)    
    PScanPP = [[[[np.zeros(maxN) for L4 in range(len(nRange))] for L3 in range(len(rRange))] for L2 in range(len(rhoRange))] for L1 in range(len(lambdaRange))]
    
#     PSurvScanPP = np.zeros(len(lambdaRange), len(rhoRange), len(rRange),
#                            len(nRange))    
    PSurvScanPP = [[[np.zeros(len(nRange)) for L3 in range(len(rRange))] for L2 in range(len(rhoRange))] for L1 in range(len(lambdaRange))]

    timePoints = range(len(t))
    nBadValues = 0

    for L1 in range(len(lambdaRange)):
#         PScanPP_local = zeros(len(rhoRange), len(rRange), len(nRange), maxN)        
        PScanPP_local = [[[np.zeros(maxN) for L4 in range(len(nRange))] for L3 in range(len(rRange))] for L2 in range(len(rhoRange))]
        
#         PSurvScanPP_local = zeros(len(rhoRange), len(rRange), len(nRange))        
        PSurvScanPP_local = [[np.zeros(len(nRange)) for L3 in range(len(rRange))] for L2 in range(len(rhoRange))]

        for L2 in range(len(rhoRange)):
            for L3 in range(len(rRange)):
                for L4 in timePoints:
                    probS = Psurv(t[L4], lambdaRange[L1], rRange[L3], rhoRange[L2])
                    avgN = 1 / probS
                    Niter = max([normal_round(int(4*avgN)), normal_round(1.5*max(nRange[L4])), 10])
                    probN = Pn(nRange[L4], t[L4], lambdaRange[L1], rRange[L3],
                               rhoRange[L2], Niter, 'returnConsecutive', maxN)
                    if any(probN > 1) or \
                            sum(probN) > 1 or \
                            probS > 1 or \
                            probS <0 or \
                            abs(probS-sum(probN))>1e-3:
                        print('Bad values at time t=' + str(t[L4]) + ' lambda=' + str(lambdaRange[L1]) + ' rho=' + str(rhoRange[L2]) + ' r=' + str(rRange[L3]))
                        print('any(probN>1=' + str(any(probN>1)) + ', sum(probN)>1=' + str(sum(probN)>1) + ', (probS-sum(probN))=' + str(probS-sum(probN)))
                        nBadValues += 1
                    else:
                        print('Completed time t=' + str(t[L4]) + ' weeks, lambda=' + str(lambdaRange[L1]) + ' rho=' + str(rhoRange[L2]) + ' r=' + str(rRange[L3]))

#                     PSurvScanPP_local[L2,L3,L4] = probS
#                     PScanPP_local[L2,L3,L4,:] = probN                  
                    PSurvScanPP_local[L2][L3][L4] = probS
                    PScanPP_local[L2][L3][L4][:] = probN
        
#         PSurvScanPP[L1,:,:,:] = PSurvScanPP_local
#         PScanPP[L1,:,:,:,:] = PScanPP_local        
        PSurvScanPP[L1] = PSurvScanPP_local
        PScanPP[L1] = PScanPP_local

#     i=len(lambdaRange)
#     j=len(rhoRange)
#     k=len(rRange)
#     l=len(t)
#     sum(PScanPP[i][j][k][l][:])/PSurvScanPP[i][j][k][l]

#     return(avgN) good
#     return(Niter) MATLAB gives 83, python gives 82
#     return(probN) small difference detected at 0.000001
#     return(probS) good
#     return(PScanPP) 
#     return(PScanPP_local) good
    return(PSurvScanPP)
#     return(PSurvScanPP_local) 
Presult = main()



Completed time t=1.4285714285714286 weeks, lambda=0.4 rho=0.5 r=0.06
Completed time t=4.285714285714286 weeks, lambda=0.4 rho=0.5 r=0.06
Completed time t=12.0 weeks, lambda=0.4 rho=0.5 r=0.06
Completed time t=26 weeks, lambda=0.4 rho=0.5 r=0.06
Completed time t=1.4285714285714286 weeks, lambda=0.4 rho=0.5 r=0.08
Completed time t=4.285714285714286 weeks, lambda=0.4 rho=0.5 r=0.08
Completed time t=12.0 weeks, lambda=0.4 rho=0.5 r=0.08
Completed time t=26 weeks, lambda=0.4 rho=0.5 r=0.08
Completed time t=1.4285714285714286 weeks, lambda=0.4 rho=0.75 r=0.06
Completed time t=4.285714285714286 weeks, lambda=0.4 rho=0.75 r=0.06
Completed time t=12.0 weeks, lambda=0.4 rho=0.75 r=0.06
Completed time t=26 weeks, lambda=0.4 rho=0.75 r=0.06
Completed time t=1.4285714285714286 weeks, lambda=0.4 rho=0.75 r=0.08
Completed time t=4.285714285714286 weeks, lambda=0.4 rho=0.75 r=0.08
Completed time t=12.0 weeks, lambda=0.4 rho=0.75 r=0.08
Completed time t=26 weeks, lambda=0.4 rho=0.75 r=0.08
Completed ti

In [9]:
Presult

[[[array([0.99775715, 0.97207092, 0.84690373, 0.66088781]),
   array([0.99701582, 0.96325704, 0.80645773, 0.59424328])],
  [array([0.9895214 , 0.93380231, 0.79644117, 0.62830432]),
   array([0.98609627, 0.913731  , 0.74590265, 0.55907856])]],
 [[array([0.99608111, 0.95703144, 0.7996882 , 0.59913025]),
   array([0.99479115, 0.94377247, 0.75031143, 0.52888567])],
  [array([0.98374203, 0.91215989, 0.75324719, 0.57221809]),
   array([0.97847142, 0.88630318, 0.69606018, 0.50083705])]]]

In [10]:
import scipy.io
MT = scipy.io.loadmat('MT5.mat')
sorted(MT.keys())

['PSurvScanPP', '__globals__', '__header__', '__version__']

In [12]:
m = MT['PSurvScanPP']
m

array([[[[0.99775715, 0.97207092, 0.84690373, 0.66088781],
         [0.99701582, 0.96325704, 0.80645773, 0.59424328]],

        [[0.9895214 , 0.93380231, 0.79644117, 0.62830432],
         [0.98609627, 0.913731  , 0.74590265, 0.55907856]]],


       [[[0.99608111, 0.95703144, 0.7996882 , 0.59913025],
         [0.99479115, 0.94377247, 0.75031143, 0.52888567]],

        [[0.98374203, 0.91215989, 0.75324719, 0.57221809],
         [0.97847142, 0.88630318, 0.69606018, 0.50083705]]]])

In [13]:
np.shape(Presult)

(2, 2, 2, 4)

In [14]:
np.shape(m)

(2, 2, 2, 4)

In [16]:
# m == Presult

In [17]:
check = m-Presult

In [18]:
TF = check < 0.000000000001

In [19]:
np.where(TF == False)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))